In [1]:
import os
import csv
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score,mean_absolute_percentage_error

### 对于DiDi，每一个n是一个模型； 对于高德，每一个pro是一个模型。每一次更新之前都要先加载模型再去训练。
### 这就意味着我们不能再进行随机的参数搜索了🤔

# 1、设定训练参数

In [2]:
# 定义训练参数
params = {
    # 基础设置
    'verbosity': 1,  # 0（静默）、1（警告）、2（信息）和 3（调试）
    'tree_method': 'hist',
    'objective': 'reg:squarederror',   # reg:pseudohubererror

    # 超参设置
    'max_depth': 6,
    'gamma': 0,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'reg_alpha': 0.05,
    'reg_lambda': 0.05,
    'n_jobs': -1,
    'seed': 42,
    'eval_metric': 'mape'
    
}

early_stopping_rounds = 10
num_boost_round = 2000
verbose_eval = 500

# 2、设定训练函数

In [3]:
def getPerformance(y_true , y_pred):
    # 计算回归性能指标
    MSE  = mean_squared_error(y_true,y_pred)
    RMSE = math.sqrt(MSE)
    mae  = mean_absolute_error(y_true,y_pred)
    r2  = r2_score(y_true,y_pred)
    mape = mean_absolute_percentage_error(y_true,y_pred)
    return {'MSE':MSE,'RMSE':RMSE,'MAE':mae,'R2':r2,'MAPE':mape}

def drawImportance(xgb_model):
    # 画出XGBoost模型的重要性
    plt.rcParams['font.sans-serif'] = ['SimHei']
    # print(xgb_model.feature_importances_)
    plot_importance(xgb_model)
    plt.show()

def train(train_df,test_df,total_test_df,features,target,objtype,n,pro,date):
    # 将df转为Dmatrix
    dtrain = xgb.DMatrix(train_df[features], label=train_df[target])
    dtest = xgb.DMatrix(test_df[features])
    total_dtest =xgb.DMatrix(total_test_df[features])

    if objtype == 'DiDi' :
        old_model_path = f'Model/DiDi_model_n{n}_date{date-1}.dat'
        new_model_path = f'Model/DiDi_model_n{n}_date{date}.dat'
    elif objtype == 'Autonavi' :
        old_model_path = f'Model/Autonavi_model_n{n}_pro{pro}_date{date-1}.dat'
        new_model_path = f'Model/Autonavi_model_n{n}_pro{pro}_date{date}.dat'

    # # 开始学习
    # if not os.path.exists(old_model_path):
    #     xgb_model = xgb.train(params, dtrain, early_stopping_rounds=early_stopping_rounds, num_boost_round=num_boost_round, verbose_eval=verbose_eval, evals=[(dtrain, 'train')])
    # else:
    #     xgb_model = xgb.train(params, dtrain, early_stopping_rounds=early_stopping_rounds, num_boost_round=num_boost_round, verbose_eval=verbose_eval, evals=[(dtrain, 'train')], xgb_model=old_model_path)
    
    xgb_model = xgb.train(params, dtrain, early_stopping_rounds=early_stopping_rounds, num_boost_round=num_boost_round, verbose_eval=verbose_eval, evals=[(dtrain, 'train')])

    # 保存模型
    xgb_model.save_model(new_model_path)

    # 进行预测
    test_df['Predicted Travel Time'] = xgb_model.predict(dtest)
    total_test_df['Predicted Travel Time'] = xgb_model.predict(total_dtest)

    performance = getPerformance(test_df[target], test_df['Predicted Travel Time'])
    total_performance = getPerformance(total_test_df[target], total_test_df['Predicted Travel Time'])

    return xgb_model,test_df,total_test_df,performance,total_performance

# 3、设定特征

In [4]:
# 设定特征
features = ['Origin','Destination','O Dig','D Dig',  # 起点与终点特征
            'Actual In Time','Hour','Quarter',   # 进入时间特征
            'Acceleration','SlowingPro',  # 车辆特征
            'PathLength','Traffic Light', # 路径特征            
            ]
target = ['Travel Time']

# 增加统计特征
def add_statistic_feature(train_df,test_df,total_test_df):
    od_feature = train_df.groupby(['OD Dig'])['Travel Time'].agg([('od_mean','mean'), ('od_max','max') , ('od_min','min') , ('od_std','std')]).reset_index()
    # odh_feature = train_df.groupby(['OD Dig','Hour'])['Travel Time'].agg([('odh_mean','mean'), ('odh_max','max') , ('odh_min','min') , ('odh_std','std')]).reset_index()
    
    train_df = pd.merge(train_df,od_feature,on='OD Dig',how='left')
    # train_df = pd.merge(train_df,odh_feature,on=['OD Dig','Hour'],how='left')

    test_df = pd.merge(test_df,od_feature,on='OD Dig',how='left')
    # test_df = pd.merge(test_df,odh_feature,on=['OD Dig','Hour'],how='left')

    total_test_df = pd.merge(total_test_df,od_feature,on='OD Dig',how='left')
    # total_test_df = pd.merge(total_test_df,odh_feature,on=['OD Dig','Hour'],how='left')

    return train_df, test_df, total_test_df, ['od_mean','od_max','od_min','od_std'] # ['od_mean','od_max','od_min','od_std','odh_mean','odh_max','odh_min','odh_std']


# 4、main函数

In [5]:
# 加载数据
n_lst = [10,20,40,60,80,100,150,200]
pro_lst = [0.001,0.005,0.01] + list(np.arange(0,0.31,0.05)[1:])
date_lst = range(2,21)


# 新建日志
log_path = "Learning_Logs/learning_logs.txt"
if not os.path.exists(log_path):
    # 如果文件不存在，则新建一个
    with open(log_path, 'w') as file:
        pass

# 新建performance
performance_path = "Performance/performance.csv"
fieldnames = ['MSE','RMSE','MAE','R2','MAPE','Type','Note','Pro','n','date']
if not os.path.exists(performance_path):
    # 如果文件不存在，则新建一个
    with open(performance_path, 'w') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        # 写入表头
        writer.writeheader()



for n in n_lst:
    
    # 读取数据
    data_df = pd.read_csv(f"Feature/VehicleData_n{n}.csv",encoding='utf-8')

    for date in date_lst:
        
        # date-1 那天用于训练，date那天用于测试

        with open(log_path, 'a') as file:
            file.write(f"-----------------------------------n : {n} ; Date: {date} ; DiDi Result:-----------------------------------\n")

        start_time = time.time()

        # 总的测试数据
        total_test_df = data_df[data_df['Date']==date]

        # DiDi数据
        DiDi_train_df = data_df[(data_df['Date']<=date-1)&(data_df['ClosedVehicle']==True)]
        DiDi_test_df  = data_df[(data_df['Date']== date )&(data_df['ClosedVehicle']==True)]

        DiDi_train_df,DiDi_test_df,DiDi_total_test_df,add_features = add_statistic_feature(DiDi_train_df,DiDi_test_df,total_test_df)
        new_features = features + add_features

        DiDi_model,DiDi_test_df,DiDi_total_test_df,DiDi_performance,DiDi_total_performance = train(DiDi_train_df,DiDi_test_df,DiDi_total_test_df,new_features,target,'DiDi',n,None,date)

        DiDi_performance['Type'],DiDi_performance['Note'],DiDi_performance['Pro'],DiDi_performance['n'],DiDi_performance['date'] = 'DiDi','pure',DiDi_train_df.shape[0]/data_df[data_df['Date']==date-1].shape[0],n,date
        DiDi_total_performance['Type'],DiDi_total_performance['Note'],DiDi_total_performance['Pro'],DiDi_total_performance['n'],DiDi_total_performance['date'] = 'DiDi','total',DiDi_train_df.shape[0]/data_df[data_df['Date']==date-1].shape[0],n,date
        
        end_time = time.time()

        with open(log_path, 'a') as file:
            file.write('pure : MAPE '+str(DiDi_performance['MAPE'])+'\n')
            file.write('total: MAPE '+str(DiDi_total_performance['MAPE'])+'\n')
            file.write(f"Time : {end_time - start_time} seconds"+'\n')

        with open(performance_path, 'a') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            # 逐行写入数据
            writer.writerow(DiDi_performance)
            writer.writerow(DiDi_total_performance)

        DiDi_test_df.to_csv(f"predictResult/DiDi_predictions_n{n}_date{date}.csv",index=None)
        DiDi_total_test_df.to_csv(f"predictResult/total_DiDi_predictions_n{n}_date{date}.csv",index=None)

        # 高德数据
        for Autonavi_pro in pro_lst:
            Autonavi_pro = round(Autonavi_pro,3)

            with open(log_path, 'a') as file:
                file.write(f"-----------------------------------n : {n} ; Date: {date} ; Autonavi {Autonavi_pro} Result:-----------------------------------\n")

            start_time = time.time()

            Autonavi_train_df = data_df[(data_df['Date']<=date-1)&(data_df['ClosedVehicle']==False)&(data_df['Tag']<=Autonavi_pro)]
            Autonavi_test_df  = data_df[(data_df['Date']== date )&(data_df['ClosedVehicle']==False)&(data_df['Tag']<=Autonavi_pro)]

            Autonavi_train_df,Autonavi_test_df,Autonavi_total_test_df,add_features = add_statistic_feature(Autonavi_train_df,Autonavi_test_df,total_test_df)
            new_features = features + add_features

            Autonavi_model,Autonavi_test_df,Autonavi_total_test_df,Autonavi_performance,Autonavi_total_performance = train(Autonavi_train_df,Autonavi_test_df,Autonavi_total_test_df,new_features,target,'Autonavi',n,Autonavi_pro,date)

            Autonavi_performance['Type'],Autonavi_performance['Note'],Autonavi_performance['Pro'],Autonavi_performance['n'],Autonavi_performance['date'] = 'Autonavi','pure',Autonavi_pro,n,date
            Autonavi_total_performance['Type'],Autonavi_total_performance['Note'],Autonavi_total_performance['Pro'],Autonavi_total_performance['n'],Autonavi_total_performance['date'] = 'Autonavi','total',Autonavi_pro,n,date

            end_time = time.time()
            
            with open(log_path, 'a') as file:
                file.write('pure : MAPE '+str(Autonavi_performance['MAPE'])+'\n')
                file.write('total: MAPE '+str(Autonavi_total_performance['MAPE'])+'\n')
                file.write(f"Time : {end_time - start_time} seconds"+'\n')

            with open(performance_path, 'a') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                # 逐行写入数据
                writer.writerow(Autonavi_performance)
                writer.writerow(Autonavi_total_performance)


            Autonavi_test_df.to_csv(f"predictResult/Autonavi_predictions_n{n}_Pro{Autonavi_pro}_date{date}.csv",index=None)
            Autonavi_total_test_df.to_csv(f"predictResult/total_Autonavi_predictions_n{n}_Pro{Autonavi_pro}_date{date}.csv",index=None)

    

[0]	train-mape:1.77451
[500]	train-mape:0.00688
[1000]	train-mape:0.00152
[1500]	train-mape:0.00036
[1999]	train-mape:0.00010


/usr/lib/python3/dist-packages/xgboost/core.py:160: UserWarning: [16:22:28] WARNING: ./src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


[0]	train-mape:2.45192
[500]	train-mape:0.00004
[1000]	train-mape:0.00002
[1106]	train-mape:0.00002


/usr/lib/python3/dist-packages/xgboost/core.py:160: UserWarning: [16:22:30] WARNING: ./src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


[0]	train-mape:2.41035
[500]	train-mape:0.00383
[1000]	train-mape:0.00031
[1500]	train-mape:0.00004
[1999]	train-mape:0.00002


...

/usr/lib/python3/dist-packages/xgboost/core.py:160: UserWarning: [23:26:54] WARNING: ./src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


[0]	train-mape:2.90763
[500]	train-mape:0.12221
[1000]	train-mape:0.10866
[1500]	train-mape:0.10225
[1999]	train-mape:0.09727


/usr/lib/python3/dist-packages/xgboost/core.py:160: UserWarning: [23:28:08] WARNING: ./src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


[0]	train-mape:2.90925
[500]	train-mape:0.12189
[1000]	train-mape:0.10916
[1500]	train-mape:0.10222
[1999]	train-mape:0.09707


/usr/lib/python3/dist-packages/xgboost/core.py:160: UserWarning: [23:29:40] WARNING: ./src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
